In [4]:
import networkx as nx
import gurobipy as gb
import pandas as pd
from scipy.spatial.distance import euclidean
from time import process_time

In [5]:
def get_graph(K,c,pi):

    V = [0] + K + [K[-1]+1]
    A = [(i,j) for i in V[:-1] for j in V[1:] if i!=j and (i,j)!=(0,V[-1])]

    rc = {arc:c[arc]-pi[arc[0]] if arc[0] != 0 else c[arc] for arc in A}
    
    return V,A,rc

def vertices_extensions(V,A):
    
    G = nx.DiGraph()
    G.add_nodes_from(V); G.add_edges_from(A)
    outbound_arcs = {}
    for v in V:
        outbound_arcs[v] = list(G.out_edges(v))
    
    return outbound_arcs

def label_algorithm(K,r,a,b,c,s,ext,mp,lbd,vehic_constr):

    def label_extension(l,arc,dominated):
        for m in range(1):
            i = arc[0]; j = arc[1]
            new_label = [[], 0, 0]

            ''' Check extension feasibility '''
            # Check for subtour
            if j in l[0]: break
            # Time windows
            if l[2]+c[i,j] > b[j]: break

            ''' Update resources consumption '''
            # Path
            new_label[0] += l[0] + [j]
            # Reduced cost
            new_label[1] = l[1] + r[(i,j)]
            # Time consumption
            new_label[2] = l[2] + c[i,j] + max(0,a[j]-l[2]-c[i,j]) + s[j]
            

            if j == K[-1]+1:
                done.append(new_label)
            else:
                new_labels[j].append(new_label)
                #label_dominance(new_label,j,dominated)
    
    def label_dominance(new_label,j,dominated):
        for l in range(len(labels[j])):
            if set(new_label[0]).issubset(labels[j][l][0]) and new_label[2]<=labels[j][l][2] and new_label[1]<=labels[j][l][1] and (len(new_label[0])<len(labels[j][l][0]) or new_label[1]<labels[j][l][1] or new_label[2]<labels[j][l][2]):
                dominated[j][l] = True

    ''' Labels dictionary '''
    # Index: number of label
    # 0: route
    # 1: cumulative reduced cost
    # 2: cumulative time consumption
    labels = dict()
    for k in K:
        if (0,k) in ext[0]: labels[k] = [ [[0,k], r[0,k], a[k]+10] ]
        else: labels[k] = []
    done = []

    act = 1
    while act > 0:
        
        L = {k:len(labels[k]) for k in K}
        new_labels = {k:[] for k in K}
        dominated = {k:{l:False for l in range(L[k])} for k in K}
        for k in K:
            for l in range(L[k]):
                if not dominated[k][l]:
                    for arc in ext[labels[k][l][0][-1]]:
                        label_extension(labels[k][l], arc, dominated)

        labels = new_labels.copy()
        act = sum(len(labels[k]) for k in K)
    
    routes = 0
    for l in range(len(done)):
        # If reduced cost is negative
        if done[l][1] < -0.001:
            col = {k:1 if k in done[l][0] else 0 for k in K}
            new_Col = gb.Column(col.values(),vehic_constr.values())
            lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=sum(c[done[l][0][k],done[l][0][k+1]] for k in range(len(done[l][0])-1)),column=new_Col))

            routes += 1

    return routes


def VRPTW_labeling(K,a,b,c,s):

    # MP model object
    mp = gb.Model("Restricted Master Problem")

    # Initial direct shipping routes for each customer
    dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=c[0,k]+c[k,K[-1]+1], name=f"DirectShip_{k}") for k in K}

    # Customers coverage constraints (i.e., each customer must be assigned to a route)
    cust_assign = {}
    for k in K:
        cust_assign[k] = mp.addConstr(dummy_0[k] == 1, f"V{k}_assignment")

    mp.setParam("OutputFlag",0)
    # lambda: variables of the routes that will be generated
    lbd = []

    # Column Generation cycle
    while True:
        
        # Solve MP
        mp.update()
        mp.optimize()

        # Get dual variables from MP
        pi = {k:cust_assign[k].getAttr("Pi") for k in K}

        ''' Solve ESPPRC with labeling algorithm '''
        # Get G=(V,A) and reduced costs of arcs
        V,A,rc = get_graph(K,c,pi)
        # Get outbound arcs of each vertex v \in V, used for label extensions
        ext = vertices_extensions(V,A)
        # Solve labeling algorithm
        opt = label_algorithm(K,rc,a,b,c,s,ext,mp,lbd,cust_assign)

        # If no columns were generated, CG is finished
        if opt == 0: break
    
    # Solve the IMP
    for v in mp.getVars():
        v.vtype = gb.GRB.BINARY
    mp.optimize()

    return mp.getObjective().getValue()



In [6]:
df = pd.read_csv("C:/Users/ari_r/Universidad de los Andes/Team site - General/2023-10/Tareas/Tarea 3/Publicar/Ruteo_drones.txt",sep="\t")
a = dict(df["READY TIME"].squeeze())
b = dict(df["DUE DATE"].squeeze())


V = list(df.index)
K = V[1:-1]

s = {v:0 if v in [0,V[-1]] else 10 for v in V}
c = {}
for i in V:
    for j in V:
        c[i,j] = euclidean([df.loc[i,"X COORD."],df.loc[i,"Y COORD."]],[df.loc[j,"X COORD."],df.loc[j,"Y COORD."]])

In [7]:
VRPTW_labeling(K,a,b,c,s)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-08


531.5386420627178